# Sensor Replay Attack Example

In [ ]:
from IPython.display import display, HTML
display(HTML('<a target="_blank" href="https://colab.research.google.com/github/https://raw.githubusercontent.com/KIOS-Research/EPyT-Flow-Dev/main/docs/examples/sensor_replay_attack.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>'))

This example demonstrates how to add a sensor replay attack to a scenario.

In [ ]:
from epyt_flow.data.benchmarks import load_leakdb_scenarios
from epyt_flow.simulation import ScenarioSimulator, SENSOR_TYPE_NODE_PRESSURE
from epyt_flow.simulation.events import SensorReplayAttack
from epyt_flow.utils import to_seconds, plot_timeseries_data

Load the first LeakDB Hanoi scenario

In [ ]:
config = load_leakdb_scenarios(scenarios_id=["1"], use_net1=False)[0]

Create new simulation

In [ ]:
sim = ScenarioSimulator(scenario_config=config)

Set simulation duration to two days

In [ ]:
sim.set_general_parameters(simulation_duration=to_seconds(days=2))

Add a sensor replay attack -- pressure readings at node "13" between 5hrs and 7hrs after simulation start (time steps 10 - 15) are replaced by the historical readings collected from the first 150min (i.e. first 5 time steps)

In [ ]:
sensor_replay_attack = SensorReplayAttack(replay_data_time_window_start=0,
                                          replay_data_time_window_end=to_seconds(minutes=150),
                                          start_time=to_seconds(hours=5),
                                          end_time=to_seconds(hours=7),
                                          sensor_id="13",
                                          sensor_type=SENSOR_TYPE_NODE_PRESSURE)

sim.add_sensor_reading_event(sensor_replay_attack)

Run simulation

In [ ]:
scada_data = sim.run_simulation()

Retrieve pressure readings

In [ ]:
pressure_readings = scada_data.get_data_pressures(sensor_locations=["13"])

In [ ]:
plot_timeseries_data(pressure_readings.T,
                     x_axis_label="Time (30min steps)",
                     y_axis_label="Pressure in $m$")

Readings for time 10 - 15 are same as the first 5 readings!

In [ ]:
plot_timeseries_data(pressure_readings[:20].T,    # The same as the first 5 readings!
                     x_axis_label="Time (30min steps)",
                     y_axis_label="Pressure in $m$")

Sensor readings WITHOUT the sensor replay attack

In [ ]:
# Remove sensor replay attack
scada_data.sensor_reading_events = []

# Recompute and show final sensor readings
pressure_readings = scada_data.get_data_pressures(sensor_locations=["13"])
plot_timeseries_data(pressure_readings[:20].T,    # The same as the first 5 readings!
                     x_axis_label="Time (30min steps)",
                     y_axis_label="Pressure in $m$")

Do not forget to close the simulation!

In [ ]:
sim.close()